In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession \
    .builder \
    .getOrCreate()

sc = spark.sparkContext

Creating RDD

In [11]:
### Create some Dump data for testing
df= spark.range (1000000)
df = df.select(df.id,df.id*2,df.id*3)
df= df.union (df)
df= df.union (df)
df= df.union (df)
df= df.union (df)
df= df.union (df)

### Convert DataFrame to RDD.
RDD = df.rdd.map (lambda x : str (x[0]) + ',' + str (x[1]) + ',' + str (x[2]))
### Save the file at a HDFS Path
RDD.coalesce(1).saveAsTextFile ('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/test_data')

In [15]:
rdd = sc.textFile('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/test_data')
rdd.getNumPartitions()

21

In [16]:
rdd.count()

32000000

In [18]:
rdd1 =rdd.filter (lambda x : int(x.split (',')[0]) ==1)
rdd1.getNumPartitions()

21

In [21]:
rdd1.count()

32

coalesce()

In [22]:
rdd1 = rdd1.coalesce(1)
rdd1.getNumPartitions()

1

Repartition(numPartitions)
- Return a new RDD that has exactly numPartitions partitions
- Create almost equal sized partitions
- Can increase or decrease the level of parallelism
- Spark performs better with equal sized partitions If you need further processing of huge data, it is preferred to have a equal sized partitions
- and so we should consider using repartition
- Internally, this uses a shuffle to redistribute data from all partitions leading to very expensive operation So avoid if not required
- If you are decreasing the number of partitions, consider using coalesce where the movement of data shuffling across the partitions is lower

In [26]:
ord= sc.textFile ('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/Orders')
#Return num records partition. This RDD have 2 partitions.
ord.glom().map(len).collect()

[34564, 34319]

In [28]:
ord = ord.repartition(5)
ord.glom().map(len).collect()

[13770, 13780, 13774, 13780, 13779]